<a href="https://colab.research.google.com/github/ToffeeNTea/CSE404-Group-Project/blob/main/CSE404_Group_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the Jupyter Notebook file for our CSE 404 Group Project. This will be run on Google Colab, and stored on Github at https://github.com/ToffeeNTea/CSE404-Group-Project.git